In [ ]:
pip install --upgrade tableauserverclient

In [38]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import flatten_dict_column, querying, flatten_dict_list_column
from tableaudocumentapi import Datasource
import tableauserverclient as TSC
import os
import time
import glob as g
from datetime import date
from urllib import parse
from tableaudocumentapi import Datasource
import tableauserverclient as TSC
import pandas as pd

In [39]:
tableau_config = {
    'tableau_online': {
        'server': 'https://10ay.online.tableau.com',
        'api_version': '3.16',
        'personal_access_token_name':'KK Token',
        'personal_access_token_secret':'<Insert Token>',
        'site_name': 'prmiinsights',
        'site_url': 'prmiinsights'
    }
}

In [40]:
conn=TableauServerConnection(tableau_config,env='tableau_online')
response=conn.sign_in()
response

<Response [200]>

In [41]:
workbook_permissions_df = pd.DataFrame()

In [42]:
workbooks_df = querying.get_workbooks_dataframe(conn)[["project", "name", "id"]]
workbooks_df = flatten_dict_column(df=workbooks_df, keys=["id", "name"], col_name="project")
workbooks_df.rename(columns={"id": "workbook_id", "name": "workbook_name"}, inplace=True)
workbooks_df.to_excel('Workbook List.xlsx')

In [43]:
workbooks_df=pd.read_excel('Production Workbook list.xlsx', sheet_name = 'Sheet2')
workbooks_df

In [45]:
groups_df = querying.get_groups_dataframe(conn)

In [46]:
for _, workbook in workbooks_df.iterrows():
    print(f"fetching group permissions for workbook '{workbook.workbook_name}...'")
    response = conn.query_workbook_permissions(workbook_id=workbook.workbook_id)
    response
    permissions_df = pd.DataFrame(response.json()["permissions"]["granteeCapabilities"])
    if "group" not in permissions_df.columns:
        continue
    permissions_df = permissions_df[permissions_df["group"].notnull()]
    permissions_df = flatten_dict_column(df=permissions_df, keys=["id"], col_name="group")
    permissions_df = flatten_dict_list_column(df=permissions_df, col_name="capabilities")
    permissions_df = flatten_dict_column(df=permissions_df, keys=["name", "mode"], col_name="capability")
    permissions_df["workbook_id"] = workbook.workbook_id
    permissions_df["workbook_name"] = workbook.workbook_name
    permissions_df["project_name"] = workbook.project_name
    permissions_df["project_id"] = workbook.project_id
    permissions_df["join_col"] = 1

    all_group_users_df = pd.DataFrame()
    group_ids = permissions_df.group_id.unique()
    for index, group_id in enumerate(group_ids):
        print(f"joining users from group '{group_id}'")
        group_users_df = querying.get_group_users_dataframe(conn=conn, group_id=group_id)
        group_users_df.rename(columns={"name": "user_name", "id": "user_id"}, inplace=True)
        group_users_df["join_col"] = index
        group_df = groups_df[groups_df["id"] == group_id].copy()[["id", "name"]]
        group_df = group_df.add_prefix("group_")
        group_df["join_col"] = index
        all_group_users_df = pd.concat([all_group_users_df, pd.merge(group_users_df, group_df, how="inner", on="join_col")])

    workbook_permissions_df = pd.concat(
        [workbook_permissions_df, permissions_df.merge(all_group_users_df, on=["group_id"])]
    )


fetching group permissions for workbook 'Suspense and PFP Scorecard...'
joining users from group 'c50b1a3e-8ee7-4352-a422-8b4567169e70'
fetching group permissions for workbook 'Post Closing Pipeline...'
joining users from group 'c50b1a3e-8ee7-4352-a422-8b4567169e70'
fetching group permissions for workbook 'Branch Rankings...'
joining users from group '4ef17401-5002-484f-9414-5d37b150a8ad'
joining users from group 'c50b1a3e-8ee7-4352-a422-8b4567169e70'
joining users from group 'eae0d2e8-d042-49f3-b0d4-e7ea0be470a5'
fetching group permissions for workbook 'GNMA Certifications Overview...'
fetching group permissions for workbook 'FHA Funded Excluding Last 6 Months...'
fetching group permissions for workbook 'Underwriting Details...'
joining users from group 'c50b1a3e-8ee7-4352-a422-8b4567169e70'
joining users from group '4ef17401-5002-484f-9414-5d37b150a8ad'
fetching group permissions for workbook 'Mid Atlantic Target Price...'
joining users from group 'c50b1a3e-8ee7-4352-a422-8b4567169e7

In [47]:
users_df = querying.get_users_dataframe(conn)
users_df.rename(columns={"id": "user_id", "name": "user_name"}, inplace=True)

In [48]:
users_df

,domain,authSetting,email,externalAuthUserId,fullName,user_id,lastLogin,user_name,siteRole,locale,language
0,{'name': 'external'},ServerDefault,tol.admin.api.broker.service.usera@tableau.com,2715380554-bec031babf6d4ac190156c2ce1dde69b,Tableau System Account,e6dcd20e-21be-4973-80fa-4a2201195722,2022-09-14T15:59:25Z,tol.admin.api.broker.service.usera@tableau.com,ServerAdministrator,,en
1,{'name': 'fe1feefe-5ee8-4e48-913b-ea491e5cf342'},SAML,mbolleddu@primeres.com,e8782e171af77c3924bab956242e203a057b7bed68ebd5...,mbolleddu@primeres.com,05c5c6c8-80eb-49cb-b4ac-ea10954e907b,2022-09-15T15:02:55Z,mbolleddu@primeres.com,SiteAdministratorCreator,,en
2,{'name': 'fe1feefe-5ee8-4e48-913b-ea491e5cf342'},SAML,kkopalley@primeres.com,312aa9432c3b7059c8e04484126f8d746f861262cad2f3...,kkopalley@primeres.com,5c5ffb75-507f-4a1e-9e57-81453f3538be,2022-09-15T15:09:40Z,kkopalley@primeres.com,SiteAdministratorCreator,,en
3,{'name': 'external'},ServerDefault,airwin@primeres.com,2228208814-a2b4aea7edb44931a719d116422c9062,Alicia Irwin,3a3ec0eb-5849-4f50-999d-4c10590efd7d,2022-09-14T17:13:43Z,airwin@primeres.com,Viewer,,en
4,{'name': 'fe1feefe-5ee8-4e48-913b-ea491e5cf342'},None,mhatch@primeres.com,7b83932c82ad9ddb368e37ff7606338fde76a385f6a457...,Mary hatch,6f87ab00-4cf5-4f97-913a-49743673a078,2022-09-01T15:53:41Z,mhatch@primeres.com,Viewer,,en
...,...,...,...,...,...,...,...,...,...,...,...
285,{'name': 'fe1feefe-5ee8-4e48-913b-ea491e5cf342'},SAML,shawn.nielsen@primeres.com,743f8622192f4bd7a5549e1deb284e9ecac300fe9552c2...,shawn.nielsen@primeres.com,900ea512-147c-4fb6-9110-149b99d953ac,2022-08-26T16:01:37Z,shawn.nielsen@primeres.com,Viewer,,en
286,{'name': 'fe1feefe-5ee8-4e48-913b-ea491e5cf342'},SAML,kathy.walker@primeres.com,330bef60a88095076981bf496a779d3f019fe6c6057d18...,kathy.walker@primeres.com,a4b5ead4-c1b3-4403-8408-770a2185bb86,NaN,kathy.walker@primeres.com,Viewer,,en
287,{'name': 'fe1feefe-5ee8-4e48-913b-ea491e5cf342'},SAML,bjacobsen@primeres.com,06b3d219305169966b9fbc323cd576bf9259e6c8c0fc9b...,bjacobsen@primeres.com,ec112c3a-9df3-41d5-9abd-53039343bffe,2022-08-31T16:54:21Z,bjacobsen@primeres.com,Viewer,,en
288,{'name': 'fe1feefe-5ee8-4e48-913b-ea491e5cf342'},SAML,lstelzer@primeres.com,011cedc821c2ba908f693e81f9fa16e9724a7b4b05a6c7...,lstelzer@primeres.com,f2ba223d-375a-4896-90c0-c25c05bacc64,2022-09-06T21:11:10Z,lstelzer@primeres.com,Viewer,,en


In [50]:
for _, workbook in workbooks_df.iterrows():
    print(f"fetching user permissions for workbook '{workbook.workbook_name}...'")
    response = conn.query_workbook_permissions(workbook_id=workbook.workbook_id)
    permissions_df = pd.DataFrame(response.json()["permissions"]["granteeCapabilities"])
    if "user" not in permissions_df.columns:
        continue
    permissions_df = permissions_df[permissions_df["user"].notnull()]
    permissions_df = flatten_dict_column(df=permissions_df, keys=["id"], col_name="user")
    permissions_df = flatten_dict_list_column(df=permissions_df, col_name="capabilities")
    permissions_df = flatten_dict_column(df=permissions_df, keys=["name", "mode"], col_name="capability")
    permissions_df["workbook_id"] = workbook.workbook_id
    permissions_df["workbook_name"] = workbook.workbook_name
    permissions_df["project_name"] = workbook.project_name
    permissions_df["project_id"] = workbook.project_id
    permissions_df["join_col"] = 1

    workbook_permissions_df = pd.concat(
        [workbook_permissions_df, permissions_df.merge(users_df, on=["user_id"])]
    )

fetching user permissions for workbook 'Suspense and PFP Scorecard...'
fetching user permissions for workbook 'Post Closing Pipeline...'
fetching user permissions for workbook 'Branch Rankings...'
fetching user permissions for workbook 'GNMA Certifications Overview...'
fetching user permissions for workbook 'FHA Funded Excluding Last 6 Months...'
fetching user permissions for workbook 'Underwriting Details...'
fetching user permissions for workbook 'Mid Atlantic Target Price...'
fetching user permissions for workbook 'Target Price & Pull Through Rate...'
fetching user permissions for workbook 'Synergy Division...'
fetching user permissions for workbook '90 Day Volume & Size...'
fetching user permissions for workbook 'Mid Atlantic Division -  LO Standings...'
fetching user permissions for workbook 'Pipeline Overview...'
fetching user permissions for workbook 'Branch Watch & Attrition Analysis...'
fetching user permissions for workbook 'Branch Scorecard...'
fetching user permissions for 

In [52]:
workbook_permissions_df.to_excel("workbook_permissions.xlsx", header=True, index=False)